## Multinomial Naive Bayes
O Multinomial Naive Bayes supõe que os recursos sejam gerados a partir de uma distribuição multinomial simples. A distribuição multinomial descreve a probabilidade de observar contagens entre várias categorias e, portanto, o Multinomial Naive Bayes é mais apropriado para características que representam contagens ou taxas de contagem (variáveis discretas).  

 A distribuição multinomial normalmente requer contagens de entidades inteiras. No entanto, na prática, contagens fracionadas como ``tf-idf`` também podem funcionar.  

O funcionamento é similar ao método gaussiano, exceto que, em vez de modelar a distribuição de dados com a distribuição normal, modelamos a distribuição de dados com uma distribuição multinomial.

http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

In [1]:
from sklearn.naive_bayes import MultinomialNB

### Exemplo 1: The 20 newsgroups  
Esse dataset compreende cerca de 18.000 postagens de grupos de notícias sobre 20 tópicos divididos em dois subconjuntos: um para treinamento (ou desenvolvimento) e outro para teste (ou para avaliação de desempenho). A divisão entre o treino e o conjunto de testes é baseada em mensagens postadas antes e depois de uma data específica.

In [2]:
# Importando os dados
from sklearn.datasets import fetch_20newsgroups

In [3]:
# Definindo as categorias 
# Em vez de utilizar as 20 categorias, escolheremos apenas 4, para rodar mais rápido
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

In [4]:
# Criando o dataset de treino com as categorias selecionadas anteriormente
train = fetch_20newsgroups(subset = 'train', categories = categories, shuffle=True)

len(train.data)  #Verificando a quantidade de linhas do dataset

2257

**Explorando o Dataset**

In [5]:
# Visualizando as classes dos 10 primeiros registros do target
for t in train.target[:10]:
    print(train.target_names[t])

comp.graphics
comp.graphics
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
sci.med
sci.med
sci.med


In [6]:
# O Scikit-Learn registra os labels como array de números, a fim de aumentar a velocidade
train.target[:10]

array([1, 1, 3, 3, 3, 3, 3, 2, 2, 2], dtype=int64)

**Preprocessamento dos dados**

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

TfidfVect = TfidfVectorizer(stop_words='english')

In [17]:
# Transformando a coleção de textos (corpus) em uma matriz esparsa
X_train = TfidfVect.fit_transform(train.data)
y_train = train.target

In [18]:
# Verficando as dimensões
print(X_train.shape, y_train.shape)

(2257, 35482) (2257,)


**Treinando o modelo**

In [19]:
MultiNB = MultinomialNB().fit(X_train, y_train)

**Fazendo previsões**

In [20]:
# Criando novos dados
docs_new = ['God is love', 'OpenGL on the GPU is fast']

In [21]:
# Transformando em metriz esparsa
x_new = TfidfVect.transform(docs_new)

In [22]:
# Fazendo a predição
predicted = MultiNB.predict(x_new)

In [23]:
#Mostrando os resultados
for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics
